In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim
from gensim import corpora,models

In [2]:
stop = stopwords.words('english')

In [3]:
a1 = pd.read_csv('content_reading1.csv',index_col=0)
a2 = pd.read_csv('content_reading2.csv',index_col=0)
a3 = pd.read_csv('content_reading3.csv',index_col=0)
a4 = pd.read_csv('content_reading4.csv',index_col=0)

In [4]:
documents = pd.concat([a1,a2,a3,a4],ignore_index=True)

In [5]:
documents['content'][0]

"There are some lovely people here who'll help you understand your reading. Here's how you can make it easier for them to help you:Share...some background and context for your reading - enough for people to understand what it's about.the question you asked, word for word.the answer Yi gave. (Our favourite format is like this: '27.2.5 to 61'.)your own feelings and thoughts about the answer.And ask for feedback!Then when you've read people's responses, please share your thoughts and update people on how it plays out. If you've browsed/searched the archives here you will know that these updates are what really make the forum a treasure - and besides, it's a kindness to the people who responded to you.Please don't miss out step 4! As Trojina said (emphasis added), Trojina said:As you say what you think and feel it opens the gates to the reading as it were. The reading is addressed to your mind so if you share your sense of the reading, the reading is actually more open for everyone to get 

In [11]:
corpus=[]
for doc in documents.content:
    text=re.sub('[^a-zA-z]',' ', doc)
    text=text.lower()
    text=text.split()
    text=[PorterStemmer().stem(word) for word in text if not word in stop]
    text=' '.join(text)
    corpus.append(text)

In [12]:
documents['clean_content'] = corpus

In [13]:
processed_docs = documents.clean_content.str.split()

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [15]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>20:
        break

0 actual
1 ad
2 address
3 answer
4 archiv
5 ask
6 aug
7 b
8 background
9 besid
10 brows
11 click
12 come
13 contact
14 context
15 easier
16 edit
17 eg
18 emphasi
19 enough
20 everyon


In [16]:
dictionary.filter_extremes(no_below=3,no_above=0.3,keep_n=1000)

In [17]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]

In [27]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 3
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=bow_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [28]:
model.log_perplexity(bow_corpus)

-6.4335224022190705

In [29]:
for idx, topic in model.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.030*"yi" + 0.023*"comment" + 0.018*"receiv" + 0.018*"answer" + 0.018*"interpret" + 0.018*"want" + 0.017*"thought" + 0.017*"move" + 0.017*"ignor" + 0.017*"correct"


Topic: 1 Word: 0.010*"hexagram" + 0.008*"want" + 0.008*"relationship" + 0.008*"say" + 0.008*"work" + 0.008*"realli" + 0.008*"seem" + 0.007*"good" + 0.007*"mean" + 0.007*"hex"


Topic: 2 Word: 0.029*"hex" + 0.029*"discuss" + 0.029*"approach" + 0.019*"imag" + 0.018*"difficulti" + 0.018*"begin" + 0.017*"right" + 0.017*"life" + 0.017*"stay" + 0.014*"superior"




In [34]:
topic=[]
for i in range(len(bow_corpus)):
    a = model[bow_corpus[i]]
    a.sort(key=lambda x: x[1],reverse=True)
    topic.append([l[0] for l in a if l[1]>0.1])

T = []
for t in topic:
    T.extend(t)


In [35]:
for i in set(T):
    print(f'Topic {i} have {T.count(i)} times')

Topic 0 have 1646 times
Topic 1 have 15973 times
Topic 2 have 917 times
